In [17]:

%reload_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import os
import pickle


from lumibot.brokers import Alpaca
from lumibot.backtesting import YahooDataBacktesting
from lumibot.strategies.strategy import Strategy
from lumibot.traders import Trader
from datetime import datetime 
from alpaca_trade_api import REST 
from timedelta import Timedelta 
from finbert_utils import estimate_sentiment
import numpy as np

from rl import DQNAgent

API_KEY = "PKJUP996961KC63A7FWO" 
API_SECRET = "FlS4JsxyehoBzODmBHBofbPyaWuqgq3QY7XyIFer" 
BASE_URL = "https://paper-api.alpaca.markets/v2"

ALPACA_CREDS = {
    "API_KEY":API_KEY, 
    "API_SECRET": API_SECRET, 
    "PAPER": True
}


In [18]:
possible_positions = ["none","buy","sell"]
market_index_symbols = ["DJI", "INX", "IXIC"]


def one_hot_encode(index, length):
    one_hot = [0] * length  # Initialize a list of 0s
    one_hot[index] = 1      # Set the value at the given index to 1
    return one_hot


In [ ]:
class NEWS:
    def __init__(self,api,start_date,end_date):
        self.start_date=start_date
        self.end_date = end_date
        self.api = api
        self.sentiment_saved = {}
        self.sentiment_read = {}
        

    def get_sentiment(self,symbol,day_range):
        filename = "saved_sentiment/"+symbol+self.start_date.strftime("%Y%m%d_")+self.end_date.strftime("%Y%m%d_")+str(day_range[0]-day_range[1])+".pkl"
        if self.sentiment_saved[filename] != "read":
            return self.sentiment_saved[filename][day_range]
        return self.read_sentiment(symbol,day_range, filename= filename)
                  
    def read_sentiment(self,symbol,day_range,filename = None):
        
        today, beggining = day_range
        news = self.api.get_news(symbol=symbol, 
                                 start=beggining, 
                                 end=today)
        
        news = [ev.__dict__["_raw"]["headline"] for ev in news]
        probability, sentiment = estimate_sentiment(news)
        if filename != None:
            self.sentiment_read[filename][day_range] = probability, one_hot_encode(sentiment,3)
        return probability, one_hot_encode(sentiment,3)
    
    

    def use_saved_sentiments(self,symbol, time_delta):
        filename = "saved_sentiment/"+symbol+self.start_date.strftime("%Y%m%d_")+self.end_date.strftime("%Y%m%d_")+str(time_delta)+".pkl"
        if os.path.exists(filename):
            with open(filename, 'rb') as file:
                data = pickle.load(file)
                self.sentiment_saved[filename] = data
        else:
            self.sentiment_saved[filename] = "read"
    
    def save_sentiments(self):
        for filename, data in self.sentiment_read.items():
            with open(filename, 'wb') as file:
                pickle.dump(data, file)
    
        
        

In [20]:

class MLTrader(Strategy): 
    def initialize(self, symbol:str="SPY", cash_at_risk:float=.5, start_date = datetime(2020,1,1), end_date = datetime(2023,12, 31), news_time_delta = Timedelta(days=3)): 
        self.symbol = symbol
        self.sleeptime = "24H" 
        self.last_trade = "none"
        
        self.cash_at_risk = cash_at_risk
        self.train = True
        self.agent = DQNAgent(22,3) #len state,len action
        self.precent_change_since_last_trade = 0
        
        
        self.money = None
        self.last_price = None
        self.quantity=None
        self.high_since_last_trade = 0
        self.low_since_last_trade = 0
        self.percent_change_since_last_trade = 0
        self.itt_number = 0
        self.last_trade_price = 100
        
        
        
        self.api = REST(base_url=BASE_URL, key_id=API_KEY, secret_key=API_SECRET)
        self.start_date = start_date
        self.end_date = end_date
        self.news_time_delta=news_time_delta
        
        self.news = NEWS(self.api,start_date,end_date)
        self.news.use_saved_sentiments(self.symbol,news_time_delta)
        for sym in market_index_symbols:
            self.news.use_saved_sentiments(sym,news_time_delta)
        

        
    def position_sizing(self): 
        cash = self.get_cash() 
        last_price = self.get_last_price(self.symbol)
        quantity = np.floor(cash * self.cash_at_risk / last_price)
        return cash, last_price, quantity

    def get_dates(self): 
        today = self.get_datetime()
        three_days_prior = today - Timedelta(days=3)
        return today.strftime('%Y-%m-%d'), three_days_prior.strftime('%Y-%m-%d')


    def get_market_sentiment(self):
        probability =[]
        sentiment = []
        for sym in market_index_symbols:
            prob, sent = self.news.get_sentiment(sym,self.get_dates())
            probability.append(prob)
            sentiment.append(sent)
        return probability, sentiment
        

        
    def get_sentiment(self): 
        return self.news.get_sentiment(self.symbol,self.get_dates())
    
    def save_sentiments(self):
        self.news.save_sentiments()
    
    
    def take_buy(self):
        
        if (self.last_trade == "sell"):
            self.take_none()
            
        if (self.last_trade == "none"):
             
                order = self.create_order(
                    self.symbol, 
                    self.quantity, 
                    "buy", 
                    type="market"
                )
                self.submit_order(order)
                
                
                self.last_trade = "buy"
                self.last_trade_price = self.last_price
                self.high_since_last_trade = self.last_price
                self.low_since_last_trade = self.last_price
                
    def take_sell(self):
        
        if (self.last_trade == "buy"):
            self.take_none()
            
        if (self.last_trade == "none"):
             
                order = self.create_order(
                    self.symbol, 
                    self.quantity, 
                    "sell", 
                    type="market"
                )
                self.submit_order(order)
                
                
                self.last_trade = "sell"
                self.last_trade_price = self.last_price
                self.high_since_last_trade = self.last_price
                self.low_since_last_trade = self.last_price
                

    def take_none(self):
        if self.last_trade == "buy" or self.last_trade == "sell":
                
                #position = self.get_position(self.symbol)
                #order = self.get_selling_order(position)
                #self.submit_order(order)
                self.sell_all()
                
                self.last_trade = "none"
                self.last_trade_price = self.last_price
                self.high_since_last_trade = self.last_price
                self.low_since_last_trade = self.last_price

                

    def store_info(self):
        
        cash, last_price, quantity = self.position_sizing()
        self.money = cash
        self.last_price = last_price
        self.quantity=quantity
        self.high_since_last_trade = max(last_price,self.last_trade_price)
        self.low_since_last_trade = min(last_price,self.last_trade_price)
        self.percent_change_since_last_trade = (self.last_price/self.last_trade_price)-1

    def reward(self):
        if self.last_trade =="sell":
            return -self.percent_change_since_last_trade
        elif self.last_trade =="buy":
            return self.percent_change_since_last_trade
        return 0
    
    
    def train_iteration(self):
        
        state = [0,0,0]
        state[possible_positions.index(self.last_trade)]=1
        
        
        probability, sentiment = self.get_sentiment()
        
        state.append(probability)
        state.extend(sentiment)
        
        
        market_prob, market_sent = self.get_market_sentiment()
        
        state.extend(market_prob)
        state.extend(market_sent)
        
        
        
        state.append(self.precent_change_since_last_trade)
        
        state.append(self.high_since_last_trade)
        state.append(self.low_since_last_trade)
        
        
        action = self.agent.act(state)
        
        next_state = state.copy()
        next_state[possible_positions.index(self.last_trade)]=0
        next_state[action]=1

        
        if possible_positions[action] == "buy":
            self.take_buy()
            
        elif possible_positions[action] == "sell":
            self.take_sell()
        else:
            self.take_none()
        
        reward = self.reward()
        
        self.agent.store_experience((state, action, reward, next_state))
        
        self.agent.train()
        
        self.itt_number += 1
        if self.itt_number % 10 == 0:
            self.agent.update_target_network()
            #print()
            #print("epsilon is currently: ", self.agent.epsilon)
            

    def run_iteration(self):
        pass
    
    
    
    def on_trading_iteration(self):
        self.store_info()
        if self.train is True:
            self.train_iteration()
            
        else:
            self.run_iteration()
        
        print(self.get_datetime())
        print(self.end_date)
        if self.get_datetime() is self.end_date:
            self.save_sentiments()
        
                


In [21]:

start_date = datetime(2020,1,1)
end_date = datetime(2023,12,31) 
broker = Alpaca(ALPACA_CREDS) 
strategy = MLTrader(name='mlstrat', broker=broker, 
                    parameters={"symbol":"SPY", 
                                "cash_at_risk":.5})

strategy.backtest(
    YahooDataBacktesting, 
    start_date, 
    end_date, 
    parameters={"symbol":"SPY", "cash_at_risk":.5}
)




# trader = Trader()
# trader.add_strategy(strategy)
# trader.run_all()


Exception in thread MLTrader:
Traceback (most recent call last):
  File "c:\Users\krist\miniconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "c:\Users\krist\miniconda3\Lib\site-packages\lumibot\strategies\strategy_executor.py", line 1069, in run
    self._initialize()
  File "c:\Users\krist\miniconda3\Lib\site-packages\lumibot\strategies\strategy_executor.py", line 335, in func_output
    result = func_input(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\krist\miniconda3\Lib\site-packages\lumibot\strategies\strategy_executor.py", line 405, in _initialize
    self.strategy.initialize(**safe_params_to_pass)
  File "C:\Users\krist\AppData\Local\Temp\ipykernel_20552\721229342.py", line 29, in initialize
TypeError: NEWS() takes no arguments


{}

In [22]:
print(torch.version.cuda)

11.8
